In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R

scratch_path <- Sys.getenv("SCRATCH")
.libPaths(file.path(scratch_path, "Rlocal4.3.2"))

In [ ]:
%%R
library(DESeq2)
library(magrittr)
library(SummarizedExperiment)

IN_DIR <- "../../inputs/"
OUT_DIR <- "../../outputs/training/"

GEO_model_training_DATA_DIR <- IN_DIR
GEO_model_training_DATA_FIL <- "rse_gene_80.Rdata"

ensembl2rxns.df <- read.table(paste(IN_DIR,"Ensembl2ReactomeReactions.txt",sep=""),
                              sep="\t")

load(paste(GEO_model_training_DATA_DIR,GEO_model_training_DATA_FIL,sep=""))

In [ ]:
%%R
tissue_data <- colData(rse_gene_80)
print(tissue_data)

In [ ]:
%%R
tissue_data <- colData(rse_gene_80)

tissue_counts <- table(tissue_data$Major_tissue)

tissues_to_remove <- names(tissue_counts[tissue_counts <= 20])

rse_gene_80 <- rse_gene_80[, !(tissue_data$Major_tissue %in% c(tissues_to_remove, 'Spleen'))]

new_tissue_counts <- table(colData(rse_gene_80)$Major_tissue)
print(new_tissue_counts)

row_totals <- rowSums(assays(rse_gene_80)$raw_counts)

rse_gene_80 <- rse_gene_80[row_totals > 0, ]

In [ ]:
%%R
new_tissue_counts_df <- as.data.frame(new_tissue_counts)
colnames(new_tissue_counts_df) <- c("Tissue", "Number")
write.csv(new_tissue_counts_df, paste0(OUT_DIR, "chart/tissue_counts.csv"), row.names = FALSE)

In [ ]:
%%R
min(rowSums(assays(rse_gene_80)$raw_count))

In [ ]:
%%R
GEO_model_training.cols <- rse_gene_80 %>% colData()
print(GEO_model_training.cols$Major_tissue)

In [ ]:
%%R
saveRDS(GEO_model_training.cols$Major_tissue,file=paste(OUT_DIR,"GEO_model_training_tissue_detail_vec.Rds",sep=""))

In [ ]:
%%R
saveRDS(GEO_model_training.cols$Row.names,file=paste(OUT_DIR,"GEO_model_training_sample_detail_vec.Rds",sep=""))

In [ ]:
%%R
GEO_model_training_df_col_data <- colData(rse_gene_80)
print(GEO_model_training_df_col_data$external_id)
sample_id_list <- GEO_model_training_df_col_data$external_id

In [ ]:
%%R
GEO_model_training.df <- rse_gene_80 %>% SummarizedExperiment::assay() %>% as.data.frame()
print(GEO_model_training.df)

In [ ]:
%%R
ensembl_wo_ids <- gsub("\\.[0-9]+","",rownames(GEO_model_training.df))
print(ensembl_wo_ids)

In [ ]:
%%R
rownames(GEO_model_training.df) <- ensembl_wo_ids
print(rownames(GEO_model_training.df))

In [ ]:
%%R
reactome_ensembl_ids <- intersect(ensembl2rxns.df$V1,ensembl_wo_ids)
print(reactome_ensembl_ids)

In [ ]:
%%R
saveRDS(reactome_ensembl_ids,file=paste(OUT_DIR,"reactome_ensembl_ids.Rds",sep=""))

In [ ]:
%%R
GEO_model_training.df <- GEO_model_training.df[reactome_ensembl_ids,]
print(GEO_model_training.df)

In [ ]:
%%R
saveRDS(GEO_model_training.df,file=paste(OUT_DIR,"GEO_model_training_df.Rds",sep=""))